In [ ]:
#импортируем все, что нужно
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D,AveragePooling2D,Conv2DTranspose, Input, Concatenate, Add, BatchNormalization, Activation, MultiHeadAttention
from tensorflow.keras.models import Model
import os
import numpy as np
import math
import tensorflow_addons as tfa
from IPython.display import clear_output
clear_output(wait=True)

#подключена ли видеокарта?
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#загрузим датасет

batch_size = 64
image_size = 64

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/users/user/8k',
    label_mode= None,
    batch_size=batch_size,
    image_size=(image_size, image_size),
    shuffle=True,
    interpolation = 'area',
    seed=123
)

#препроцессинг
@tf.function
def process(x):
    #приводим значения пикселей к [0...1]
    return (x/256.0)

dataset = dataset.map(process)
dataset = dataset.shuffle(500).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#посмотрим на картинки
def imshow(): 
    n = 5
    plt.figure(figsize=(10, 6))
    #берем один батч из датасета. проходимся по первым n
    for images in dataset.take(1):
        for i in range(n):
            img = images[i]
            ax = plt.subplot(3, n, i + 1 + n)
            
            plt.imshow(img, cmap='gist_gray')
         #   plt.axis('off')
            ax.get_yaxis().set_visible(False)           
    plt.show()
imshow()



In [ ]:
#определим некоторые параметры
min_signal_rate = 0.02
max_signal_rate = 0.95


In [ ]:
#как представить данные о времени для сверточного слоя?
#лучший способ - sinusoidal embedding

embedding_dims = 32
embedding_max_frequency = 1000.0

def sinusoidal_embedding(x):
    embedding_min_frequency = 1.0
    frequencies = tf.exp(
        tf.linspace(
            tf.math.log(embedding_min_frequency),
            tf.math.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = 2.0 * math.pi * frequencies
    embeddings = tf.concat(
        [tf.sin(angular_speeds * x), tf.cos(angular_speeds * x)], axis=3
    )
    return embeddings

In [ ]:
#немного визуализации
example_time = tf.constant([[[[0.3]]]])

print(sinusoidal_embedding(example_time).shape)

#найдем крайние углы
start_angle = tf.acos(max_signal_rate)
end_angle = tf.acos(min_signal_rate)

example_batch_of_times = tf.range(start_angle, end_angle, delta=0.01)

#добавим несколько измерений, чтобы функция заработала
for _ in range(3):
    example_batch_of_times = tf.expand_dims(example_batch_of_times, axis = -1)


example_embeddings = sinusoidal_embedding(example_batch_of_times)

print(example_embeddings.shape)

#обратно для отображения
example_embeddings = tf.squeeze(example_embeddings, axis = [1,2])

example_embeddings = example_embeddings.numpy()

#отрисовываем
plt.figure(figsize=(10, 6))
plt.imshow(example_embeddings)
plt.show()

In [ ]:
#код сборки нейросети

widths = [32, 64, 96, 128, 256]
block_depth = 2

#основной "вычислительный" блок, на них приходится большинство параметров

def ResidualBlock(width):
    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv2D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same", activation=keras.activations.swish)(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply

#есть два варианта блока на выбор. Работают вроде одинаково

'''
def ResidualBlock(width):
    def apply(x): 
        
        conv1 = Conv2D(width, 3, activation = 'relu', padding = 'same')(x)
        convint = Conv2D(width, 3, padding = 'same')(conv1)
        convint = BatchNormalization()(convint)
        convint = Activation('relu')(convint)
        convint = Conv2D(width, 3, padding = 'same')(convint)
        convint = BatchNormalization()(convint)
        summ = Add()([convint, conv1])
        x = Activation('relu')(summ)
        return x
    return apply
'''

#понижающий блок
def DownBlock(width, block_depth):
    def apply(x):
        x, skips, emb = x
        height = x.shape[1]
        e = layers.UpSampling2D(size=height, interpolation="nearest")(emb)
        x = Concatenate()([x, e])
        for _ in range(block_depth):
            x = ResidualBlock(width)(x)
            skips.append(x)
        x = layers.AveragePooling2D(pool_size=2)(x)
        return x

    return apply

#повышающий блок
def UpBlock(width, block_depth):
    def apply(x):
        x, skips = x
        x = layers.UpSampling2D(size=2, interpolation="bilinear")(x)
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x)
        return x

    return apply


def get_network(image_size, widths, block_depth):
    noisy_images = keras.Input(shape=(image_size, image_size, 3))
    noise_variances = keras.Input(shape=(1, 1, 1))
    
    x = noisy_images
    emb = layers.Lambda(sinusoidal_embedding)(noise_variances)

    skips = []
    
    #сборка нейросети U-net автоматическая. Имеет смысл менять только кол-во блоков и
    #архитектуру основного блока. Остальное код сделает за нас)
    
    for width in widths[:-1]:
        x = DownBlock(width, block_depth)([x, skips, emb])

    for _ in range(block_depth):
        x = ResidualBlock(widths[-1])(x)


    for width in reversed(widths[:-1]):
        x = UpBlock(width, block_depth)([x, skips])
        
    #для лучшей четкости картинки 
    e = layers.UpSampling2D(size=image_size, interpolation="nearest")(emb)
    x = layers.Concatenate()([x, noisy_images, e])
    x = layers.Conv2D(32, kernel_size=1, padding = 'same', activation=keras.activations.swish )(x)

    x = layers.Conv2D(3, kernel_size=1,  kernel_initializer="zeros")(x)

    return keras.Model([noisy_images, noise_variances], x, name="residual_unet")

In [ ]:
class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, block_depth):
        super().__init__()

        self.normalizer = layers.Normalization()
        self.network = get_network(image_size, widths, block_depth)
        
        self.optimizer = optimizer=tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)
    

    def denormalize(self, images):
        #обратная задача нормализации
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return tf.clip_by_value(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        
        #найдем такой угол, для которого cos будет min и max т.е. 0 и 1
        start_angle = tf.acos(max_signal_rate)
        end_angle = tf.acos(min_signal_rate)

        #переменная diffusion_times сторого от 0 до 1 линейно сдвигает угол от start к end
        diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

        #считаем параметры, основываясь на угол (а по сути на diffusion_times)
        signal_rates = tf.cos(diffusion_angles)
        noise_rates = tf.sin(diffusion_angles)
        # косинусы с синусами не просто так: sin^2(x) + cos^2(x) = 1

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates):
        
        #предсказавыем весь шум, предсказанный нейросетью
        pred_noises = self.network([noisy_images, noise_rates**2])
        
        #и находим чистые данные, удаляя шум так, будто шаг всего один
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps):
        
        num_images = initial_noise.shape[0]
        #найдем размер шага
        step_size = 1.0 / diffusion_steps

        # на первом шаге "noisy image" чистый шум
        # но его signal rate должен быть ненулевым (min_signal_rate)
        
        next_noisy_images = initial_noise
        
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images
            
            #время диффузии идет в обратную сторону (1 - step)
            diffusion_times = tf.ones((num_images, 1, 1, 1)) - step * step_size
            
            #находим параметры
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            
            #предсказываем шум и сигнал без шума, 'разделяем' их
            pred_noises, pred_images = self.denoise(noisy_images, noise_rates, signal_rates)
           
            #если отнять stepsize, получим время для следующего шага
            next_diffusion_times = diffusion_times - step_size
            
            #считаем параметры (теперь значение шума меньше, значение сигнала больше)
            next_noise_rates, next_signal_rates = self.diffusion_schedule(next_diffusion_times)
            
            #и с новыми параметрами опять собираем зашумленный сигнал
            #из того-же шума и того-же сигнала
            next_noisy_images = (next_signal_rates * pred_images + next_noise_rates * pred_noises)

        return pred_images

    def generate(self, num_images, diffusion_steps):
        
        initial_noise = tf.random.normal(shape=(num_images, image_size, image_size, 3))
        generated_images = self.reverse_diffusion(initial_noise, diffusion_steps)
        generated_images = self.denormalize(generated_images)
        return generated_images
    
    @tf.function
    def train_step(self, images):
        
        images = self.normalizer(images, training=True)
        
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        
        diffusion_times = tf.random.uniform(shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0)
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        
        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            
            pred_noises, pred_images = self.denoise(noisy_images, noise_rates, signal_rates)
            
            noise_loss = tf.reduce_mean(tf.abs(noises - pred_noises), axis = [1,2,3])

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))
       
        return noise_loss

    def plot_images(self, epoch=None, logs=None, num_rows=3, num_cols=6):
        
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=plot_diffusion_steps,
        )

        plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
        for row in range(num_rows):
            for col in range(num_cols):
                index = row * num_cols + col
                plt.subplot(num_rows, num_cols, index + 1)
                plt.imshow(generated_images[index])
                plt.axis("off")
        plt.tight_layout()
        plt.show()
        plt.close()
        
model = DiffusionModel(image_size, widths, block_depth)


In [ ]:
#инициализация нормализатора, всегда выполнять перед обучением
for a in dataset.take(10):
    model.normalizer.adapt(a)

In [ ]:
plot_diffusion_steps = 100
model.plot_images()

In [ ]:
#обучаем
epochs = 50
hist = np.array(np.empty([0]))
from IPython.display import clear_output
for epoch in range(epochs):
    

   
    midloss = 0
    for step, x in enumerate(dataset):
        if tf.shape(x)[0] == 64: #проверяем целостность батча
            midloss += tf.reduce_mean(model.train_step(x), axis = 0)

        if(step == 5):  
            clear_output(wait=True)
            print('эпоха ' + str(epoch))
            print('ошибка: ' + str(float(midloss/5)))
           
            hist = np.append(hist, float(midloss/5))
            plt.plot(np.arange(0,len(hist)), hist)
            plt.show()
            #model.plot_images()